# Census Transit Mapping - Data cleaning

This data is sourced from the Canadian Census. 

First, I uploaded the census tract map file for the Winnipeg CMA to QGIS and removed tracts outside the actual city of Winnipeg bounds (transit does not apply outside the city). I then downloaded 2021 census-tract level data regarding:

    - percentage of population commuting by transit
    - 2020 poverty rate (mbm)
    - proportion of population considered low income
    - population density per square km
    - population change between census years (2016-2021)

In this notebook, I will merge the columns with relevant data from the above sheet to the map file containing geometry in order to create choropleth maps for analysis.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in geometry file
tracts = pd.read_csv('winnipeg-census-tracts.csv')

In [4]:
tracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   CTUID     170 non-null    float64
 1   DGUID     170 non-null    object 
 2   CTNAME    170 non-null    float64
 3   LANDAREA  170 non-null    float64
 4   PRUID     170 non-null    int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 6.8+ KB


In [1]:
! ls

census-mapping-transit.ipynb
census-tract-data-clean.csv
Percentage-of-commuters-public-transit.csv
Population-density-per-square-kilometre.csv
Population-percentage-change.csv
Poverty-rate-2020-mbm.csv
Prevalence-low-income-2020-limat.csv
winnipeg-census-tracts.csv


In [40]:
tracts.head()

,CTUID,DGUID,CTNAME,LANDAREA,PRUID
0,6020520.05,2021S05076020520.05,520.05,26.3221,46
1,6020001.00,2021S05076020001.00,1.00,1.7766,46
2,6020002.00,2021S05076020002.00,2.00,1.8471,46
3,6020003.00,2021S05076020003.00,3.00,1.4086,46
4,6020005.00,2021S05076020005.00,5.00,2.3718,46


In [59]:
# Read in pop density

popden = pd.read_csv('Population-density-per-square-kilometre.csv')

In [42]:
popden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 3 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   DGUID                                    187 non-null    object
 1   Location                                 187 non-null    object
 2   Population density per square kilometre  187 non-null    object
dtypes: object(3)
memory usage: 4.5+ KB


There are 17 extraneous locations.

**DGUID** is a unique code for each census area. [See documentation.](https://www12.statcan.gc.ca/census-recensement/2021/ref/dict/az/definition-eng.cfm?ID=geo055) It will be the perfect variable to match when merging frames.

In [43]:
popden.head()

,DGUID,Location,Population density per square kilometre
0,2021S05076020023.00,6020023.00 [CT] (Man.),"12,207.4"
1,2021S05076020011.00,6020011.00 [CT] (Man.),"10,695.8"
2,2021S05076020014.00,6020014.00 [CT] (Man.),"9,615.8"
3,2021S05076020028.00,6020028.00 [CT] (Man.),"7,971.7"
4,2021S05076020022.00,6020022.00 [CT] (Man.),"7,710.5"


In [44]:
# First experiment, then create a function:

pd.merge(tracts,popden, on='DGUID')

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,Location,Population density per square kilometre
0,6020520.05,2021S05076020520.05,520.05,26.3221,46,6020520.05 [CT] (Man.),27.5
1,6020001.00,2021S05076020001.00,1.00,1.7766,46,6020001.00 [CT] (Man.),"2,371.4"
2,6020002.00,2021S05076020002.00,2.00,1.8471,46,6020002.00 [CT] (Man.),"2,999.3"
3,6020003.00,2021S05076020003.00,3.00,1.4086,46,6020003.00 [CT] (Man.),"3,657.5"
4,6020005.00,2021S05076020005.00,5.00,2.3718,46,6020005.00 [CT] (Man.),"2,323.6"
...,...,...,...,...,...,...,...
165,6020100.10,2021S05076020100.10,100.10,1.5457,46,6020100.10 [CT] (Man.),"3,719.4"
166,6020500.10,2021S05076020500.10,500.10,1.8869,46,6020500.10 [CT] (Man.),"3,290.1"
167,6020500.11,2021S05076020500.11,500.11,3.7967,46,6020500.11 [CT] (Man.),"1,815.5"
168,6020500.12,2021S05076020500.12,500.12,4.0166,46,6020500.12 [CT] (Man.),"1,826.4"


In [45]:
# This will work as long as I drop the location column. 
# Let's import the other data and make sure that's consistent.

transit=pd.read_csv('Percentage-of-commuters-public-transit.csv')
transit.head()

,DGUID,Location,Percentage of commuters: public transit
0,2021S05076020501.01,6020501.01 [CT] (Man.),36.9
1,2021S05076020023.00,6020023.00 [CT] (Man.),34.3
2,2021S05076020022.00,6020022.00 [CT] (Man.),32.1
3,2021S05076020013.00,6020013.00 [CT] (Man.),30.3
4,2021S05076020015.00,6020015.00 [CT] (Man.),29.5


In [46]:
popchange=pd.read_csv('Population-percentage-change.csv')
popchange.head()

,DGUID,Location,"Population percentage change, 2016 to 2021"
0,2021S05076020800.00,6020800.00 [CT] (Man.),29.0
1,2021S05076020700.00,6020700.00 [CT] (Man.),13.4
2,2021S05076020600.00,6020600.00 [CT] (Man.),11.8
3,2021S05076020595.02,6020595.02 [CT] (Man.),\t-1.3
4,2021S05076020595.01,6020595.01 [CT] (Man.),5.0


In [47]:
mbm=pd.read_csv('Poverty-rate-2020-mbm.csv')
mbm.head()

,DGUID,Location,Poverty rate in 2020 (MBM) (%)
0,2021S05076020501.01,6020501.01 [CT] (Man.),34.0
1,2021S05076020022.00,6020022.00 [CT] (Man.),32.9
2,2021S05076020015.00,6020015.00 [CT] (Man.),30.8
3,2021S05076020023.00,6020023.00 [CT] (Man.),28.2
4,2021S05076020025.00,6020025.00 [CT] (Man.),27.5


In [48]:
limat=pd.read_csv('Prevalence-low-income-2020-limat.csv')
limat.head()

,DGUID,Location,Prevalence of low income in 2020 (LIM-AT) (%)
0,2021S05076020034.00,6020034.00 [CT] (Man.),44.4
1,2021S05076020025.00,6020025.00 [CT] (Man.),43.7
2,2021S05076020026.00,6020026.00 [CT] (Man.),41.6
3,2021S05076020022.00,6020022.00 [CT] (Man.),40.2
4,2021S05076020023.00,6020023.00 [CT] (Man.),37.6


In [56]:
# Great, let's define a function that will do the following:
# Given a dataframe, drop the column called 'Location'
# Merge with the tracts frame on the DGUID

def census_merge(data, census):
    data.drop(columns='Location',inplace=True)
    census = pd.merge(census, data, on='DGUID')
    return census

In [60]:
# Now let's try it

tracts = census_merge(popden, tracts)

In [62]:
# Repeat with other vars

tracts = census_merge(transit,tracts)
tracts = census_merge(popchange,tracts)
tracts = census_merge(mbm,tracts)
tracts = census_merge(limat,tracts)

In [63]:
tracts

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,Population density per square kilometre,Percentage of commuters: public transit,"Population percentage change, 2016 to 2021",Poverty rate in 2020 (MBM) (%),Prevalence of low income in 2020 (LIM-AT) (%)
0,6020520.05,2021S05076020520.05,520.05,26.3221,46,27.5,0.0,8.5,0.0,5.5
1,6020001.00,2021S05076020001.00,1.00,1.7766,46,"2,371.4",7.8,\t-1.5,6.2,7.4
2,6020002.00,2021S05076020002.00,2.00,1.8471,46,"2,999.3",12.7,5.8,8.5,13.9
3,6020003.00,2021S05076020003.00,3.00,1.4086,46,"3,657.5",14.5,\t-0.2,11.8,13.4
4,6020005.00,2021S05076020005.00,5.00,2.3718,46,"2,323.6",3.8,\t-4.4,1.7,3.6
...,...,...,...,...,...,...,...,...,...,...
165,6020100.10,2021S05076020100.10,100.10,1.5457,46,"3,719.4",3.8,\t-2.7,3.0,5.1
166,6020500.10,2021S05076020500.10,500.10,1.8869,46,"3,290.1",7.1,\t-0.9,21.5,19.7
167,6020500.11,2021S05076020500.11,500.11,3.7967,46,"1,815.5",3.8,8.2,12.7,15.4
168,6020500.12,2021S05076020500.12,500.12,4.0166,46,"1,826.4",5.8,52.1,13.1,15.6


In [82]:
# Nice! 
# Now some cleaning

tracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CTUID          170 non-null    float64
 1   DGUID          170 non-null    object 
 2   CTNAME         170 non-null    float64
 3   LANDAREA       170 non-null    float64
 4   PRUID          170 non-null    int64  
 5   pop_density    170 non-null    float64
 6   transit_users  169 non-null    float64
 7   pop_change     170 non-null    object 
 8   poverty_rate   168 non-null    float64
 9   low_income     168 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 13.4+ KB


In [67]:
# Rename columns

tracts.rename(columns={'Population density per square kilometre': 'pop_density', 
                       'Percentage of commuters: public transit': 'transit_users', 
                       'Population percentage change, 2016 to 2021':'pop_change',
                       'Poverty rate in 2020 (MBM) (%)': 'poverty_rate',
                       'Prevalence of low income in 2020 (LIM-AT) (%)':'low_income'
                      }, inplace=True)

In [78]:
# There are some values in low_income, transit_users and poverty_rate written as 'x' — they should be null
# Change x to null, then convert to float

tracts['low_income'] = tracts['low_income'].replace('x', None)
tracts['low_income'] = tracts['low_income'].astype(float)

tracts['transit_users'] = tracts['transit_users'].replace('x', None)
tracts['transit_users'] = tracts['transit_users'].astype(float)

tracts['poverty_rate'] = tracts['poverty_rate'].replace('x', None)
tracts['poverty_rate'] = tracts['poverty_rate'].astype(float)

In [80]:
# Check pop_density for x values

tracts[tracts['pop_density']=='x']

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,pop_density,transit_users,pop_change,poverty_rate,low_income


In [81]:
# Convert pop density to float by removing comma and setting as float

tracts['pop_density']=tracts['pop_density'].str.replace(',','')
tracts['pop_density'] = tracts['pop_density'].astype(float)

In [86]:
# Now we have pop_change, which contains negative values with a \t in front
# Easy enough to strip and convert to float

tracts['pop_change'] = tracts['pop_change'].str.strip('\t')

In [96]:
# Check for weird null values
tracts[tracts['pop_change']=='…']
# Damn what even is that...
tracts['pop_change'] = tracts['pop_change'].replace('…', None)


In [97]:
tracts['pop_change'] = tracts['pop_change'].astype(float)

In [98]:
tracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CTUID          170 non-null    float64
 1   DGUID          170 non-null    object 
 2   CTNAME         170 non-null    float64
 3   LANDAREA       170 non-null    float64
 4   PRUID          170 non-null    int64  
 5   pop_density    170 non-null    float64
 6   transit_users  169 non-null    float64
 7   pop_change     169 non-null    float64
 8   poverty_rate   168 non-null    float64
 9   low_income     168 non-null    float64
dtypes: float64(8), int64(1), object(1)
memory usage: 13.4+ KB


In [99]:
# Excellent, now let's return this as a csv so it can be merged with maps!

tracts.to_csv('census-tract-data-clean.csv')